In [1]:
import tensorflow as tf
import numpy as np
import random

In [53]:
GEN_RANK_NUM = 4+1
LENGTH = 7
batch_size = 3
K = 1

In [68]:
np.random.seed(1)
a = np.random.random_sample((batch_size, LENGTH,GEN_RANK_NUM))
#a = np.reshape(a,[3,70*5])
tensor_a = tf.convert_to_tensor(a)
a[0]

array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
        1.46755891e-01],
       [9.23385948e-02, 1.86260211e-01, 3.45560727e-01, 3.96767474e-01,
        5.38816734e-01],
       [4.19194514e-01, 6.85219500e-01, 2.04452250e-01, 8.78117436e-01,
        2.73875932e-02],
       [6.70467510e-01, 4.17304802e-01, 5.58689828e-01, 1.40386939e-01,
        1.98101489e-01],
       [8.00744569e-01, 9.68261576e-01, 3.13424178e-01, 6.92322616e-01,
        8.76389152e-01],
       [8.94606664e-01, 8.50442114e-02, 3.90547832e-02, 1.69830420e-01,
        8.78142503e-01],
       [9.83468338e-02, 4.21107625e-01, 9.57889530e-01, 5.33165285e-01,
        6.91877114e-01]])

In [69]:
tensor_a.shape[0]

Dimension(3)

In [70]:
tf.shape(tensor_a)[0]

<tf.Tensor 'strided_slice_25:0' shape=() dtype=int32>

In [71]:
check = []
check_tb = []

In [72]:
def set_value(matrix, x, y, col_len=4,val=0.):
    # 提取出要更新的行
    row = tf.gather(matrix, x)
    # 构造这行的新数据
    new_row = tf.concat([row[:y], [val]*(col_len-y)], axis=0)
    # 使用 tf.scatter_update 方法进正行替换
    return tf.scatter_update(matrix, x, new_row)  # matrix必须是var

In [73]:
def set_col(matrix,new_col,col):
    #return tf.transpose(tf.scatter_update(tf.transpose(matrix),col,tf.Variable(new_col)))  # 
    print('matrix[:,:col]',matrix[:,:col])
    print('tf.reshape(new_col,[LENGTH,1])',tf.reshape(new_col,[LENGTH,1]))
    print('matrix[:,col+1:]]',matrix[:,col+1:])
    return tf.concat([matrix[:,:col],tf.reshape(new_col,[LENGTH,1]),matrix[:,col+1:]],1)  # 导致下一步报错
    #return tf.assign(tf.slice(matrix,[0,col],[LENGTH,1]),new_col)  #ValueError: Sliced assignment is only supported for variables
   
    

In [75]:
def tf_check_repeat(checked, cand_set):
    flag = tf.convert_to_tensor(False)
    
    for cand_tensor in cand_set:
        eq = tf.equal(checked,cand_tensor)
        cond = tf.equal(tf.reduce_sum(tf.cast(eq,tf.int32)),GEN_RANK_NUM-1)
        check.append((cond,eq,tf.convert_to_tensor(flag),tf.equal(checked,cand_tensor)))
        flag = tf.cond(cond, lambda: tf.cast(True, tf.bool), lambda: flag)
    return flag  # 但凡有一个重复则true

In [76]:
def tf_sample_and_test(prob_table, temperature, k=K):
    prob_table = tf.convert_to_tensor(prob_table)
    if len(prob_table.shape) == 3:
        prob_table = prob_table[0:,0:,:-1]
    elif len(prob_table.shape) == 2:
        prob_table = np.reshape(prob_table, (-1, LENGTH, GEN_RANK_NUM))[:,:,0:-1]   

    prob_table = tf.math.exp(prob_table * temperature)  # [bs, LENGTH, GEN_RANK_NUM-1]  [3,70,4]
    
    # 从每条候选概率[70,4]中抽样20个即[20,4]
    i = tf.constant(0,name='cons_i')
    prob_table_frame = tf.Variable(prob_table[0],name='prob_frame')
    sample_res = tf.Variable([],name='sample_res',dtype=tf.int64)
    def condition(i, prob_table,_):
        return i<tf.shape(prob_table)[0]
    
    def loop_body(i, prob_table,res):

        sample_set = []
        num = k
        cnt = 0
        while num > 0:  # 抽取20个sku的位置概率,K=2
            tf.add_to_collection('i', value=[i])
            cnt += 1
            print('num:',num)
            print('count:',cnt)
            #init = prob_table[i]
            #table  = tf.Variable(init)  # 每次都从原始的数据开始
            table = tf.assign(prob_table_frame, prob_table[i])
            #check_tb.append(table)
            print('table:',table)
            tmp = tf_sample_one(table)  # [70,4] -> [1,4]
            if len(sample_set)==0:
                sample_set_concat = tmp
                sample_set.append(tmp)
                num -= 1
            else:
                tmp_len = len(sample_set)#sample_set_concat.get_shape().as_list()[0]
                sample_set_concat = tf.cond(tf_check_repeat(tmp,sample_set), 
                        true_fn=lambda: sample_set_concat, 
                        false_fn=lambda: tf.concat([sample_set_concat,tmp],0))  
                if sample_set_concat.get_shape().as_list()[0]!=tmp_len:
                    sample_set.append(tmp)
                    num -= 1

            if sample_set_concat.get_shape().as_list()[0] == k:
                break
        tmpp = tf.reshape(sample_set_concat,[k*(GEN_RANK_NUM-1),])
        res = tf.concat([res, tmpp], 0)

        i += 1
        return [i,prob_table,res]
        
    _i, _prob_table,res = tf.while_loop(condition, 
                                        loop_body, 
                                        [i, prob_table, sample_res],
                                        shape_invariants=[i.get_shape(),
                                                          prob_table.get_shape(),
                                                          tf.TensorShape([None])])
    return res  # [bs, K, GEN_RANK_NUM-1]

In [85]:
def tf_sample_one(table):
    np.random.seed(16)
    random_shifts = np.random.random([LENGTH,GEN_RANK_NUM-1])  # [70,4]
    #print('------------\n',random_shifts,'-----------\n')
    random_shifts /= random_shifts.sum(axis = 0)[np.newaxis,:]
    result = []

    for m in range(GEN_RANK_NUM-1):  # 4个位置
        table_m = table[:,m]  # [70,1]

        table_m /= tf.reduce_sum(table_m)  # 这里要控制执行顺序，先计算这里，再置零
        shifted_probabilities = random_shifts[:,m] - table_m
        with tf.control_dependencies([table_m,shifted_probabilities]):

            l = tf.argmin(shifted_probabilities)  # 找出最小的index
            table = set_value(table,x=l,y=m,col_len=GEN_RANK_NUM-1,val=0.)
            result.append(l)

    return tf.reshape(result,[1,len(result)])

In [86]:
check = []
check_tb = []


In [87]:

tf_k_list = tf_sample_and_test(a, 0.4)
tf_k_list

num: 1
count: 1
table: Tensor("while_13/Assign:0", shape=(7, 4), dtype=float64_ref)


<tf.Tensor 'while_13/Exit_2:0' shape=(?,) dtype=int64>

In [88]:
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [89]:
k_list=sess.run([tf.reshape(tf_k_list,[-1,K,4])])

In [90]:
k_list

[array([[[5, 6, 3, 0]],
 
        [[2, 6, 3, 0]],
 
        [[2, 1, 3, 0]]], dtype=int64)]

In [84]:
k_list[0].shape

(3, 1, 4)

In [66]:
def tf_sample_and_test(prob_table, temperature, k=K):
    prob_table = tf.convert_to_tensor(prob_table)
    if len(prob_table.shape) == 3:
        prob_table = prob_table[0:,0:,:-1]
    elif len(prob_table.shape) == 2:
        prob_table = np.reshape(prob_table, (-1, LENGTH, GEN_RANK_NUM))[:,:,0:-1]   
    res = []
    prob_table = tf.math.exp(prob_table * temperature)  # [bs, LENGTH, GEN_RANK_NUM-1]  [3,70,4]
    
    # 从每条候选概率[70,4]中抽样20个即[20,4]
    i = 0
    for i in range(prob_table.get_shape().as_list()[0]):  # bs
        print('exam:',i)
        sample_set = []  # 对于每个样本生成k个list
        print('k={}'.format(k))
        num = k
        cnt = 0
        while num > 0:  # 抽取20个sku的位置概率
            cnt += 1
            print('count:',cnt)
            table  = tf.Variable(prob_table[i])  # 每次都从原始的数据开始
            #check_tb.append(table)
            print('table:',table)
            tmp = tf_sample_one(table)  # [70,4] -> [1,4]
            if len(sample_set)==0:
                sample_set_concat = tmp
                sample_set.append(tmp)
                num -= 1
                print('第一次加入sample-set list')
            else:
                print('进入非第一次')
                tmp_len = len(sample_set)#sample_set_concat.get_shape().as_list()[0]
                sample_set_concat = tf.cond(tf_check_repeat(tmp,sample_set), 
                        true_fn=lambda: sample_set_concat, 
                        false_fn=lambda: tf.concat([sample_set_concat,tmp],0))  
                if sample_set_concat.get_shape().as_list()[0]!=tmp_len:
                    sample_set.append(tmp)
                    num -= 1
                    print('不通的list入栈')
                #sample_set.append(tmp)
                print('[!] tmp:',tmp)
                print('[!] sample_set_concat:',sample_set_concat)
                check.append((tmp,sample_set_concat))
            if sample_set_concat.get_shape().as_list()[0] == k:
                break
            if cnt > 3:
                break
        res.append(sample_set_concat)
    print('[#] finish',res)
    res = tf.stack(res,0)
    return res  # [bs, K, GEN_RANK_NUM-1]